#Llama 2: Leveraging the META Language Model on HuggingFace

Explore the capabilities of the META LLM (Language Model) and its integration with HuggingFace for innovative natural language processing tasks and applications. Join us in harnessing the power of cutting-edge AI for text generation and understanding.

In [ ]:
import torch

# Check if a GPU is available
if torch.cuda.is_available():
    # Get the name of the GPU
    gpu_name = torch.cuda.get_device_name(0)

    # Get the GPU's memory capacity
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3)  # in GB

    print(f"GPU Name: {gpu_name}")
    print(f"GPU Memory Capacity: {gpu_memory} GB")
else:
    print("No GPU available.")

GPU Name: Tesla T4
GPU Memory Capacity: 14.74786376953125 GB


## Install libraries

In [ ]:
#to use the model locally
!pip install -qU transformers accelerate einops langchain xformers bitsandbytes faiss-gpu sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 62.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.6/211.6 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 118.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.5 MB/s eta 0:00:00


In [ ]:
#display information about the NVIDIA GPUs installed on your system
!nvidia-smi

Fri Sep 29 12:06:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P8     9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Get access to Huggingface

In [ ]:
#https://huggingface.co/docs/api-inference/quicktour#get-your-api-token

from getpass import getpass

HUGGINGFACEHUB_API_TOKEN = getpass()

··········


This code segment performs the following tasks:

1. Imports the necessary libraries, including cuda (for GPU operations), bfloat16 (a data type for GPU optimization), and transformers (for working with pre-trained language models).

2. Defines the model_id, which specifies the identifier for a pre-trained language model.

3. Determines the device for model execution based on GPU availability. If a GPU is available, it sets the device to be used; otherwise, it falls back to using the CPU.

4. Configures quantization settings using the bitsandbytes library. Quantization is a technique used to reduce the memory and computational requirements of the model.

5. Initializes items related to the Hugging Face (HF) ecosystem, such as authentication using an access token, model configuration, and loads a pre-trained model for causal language modeling.

6. Sets the model in evaluation mode, enabling it for inference.

In summary, this code prepares a pre-trained language model for usage, optimizes it for GPU memory usage through quantization, and ensures it's ready for evaluation and inference tasks.

In [ ]:
from torch import cuda, bfloat16
import transformers

model_id = 'meta-llama/Llama-2-13b-chat-hf'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, you need an access token
hf_auth = HUGGINGFACEHUB_API_TOKEN #'<add your access token here>'
#create a model configuration object
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)

# enable evaluation mode to allow model inference (not update the weights)
model.eval()

print(f"Model loaded on {device}")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:1006: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/usr/local/lib/python3.10/dist-packages/transformers/models/auto/auto_factory.py:479: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Model loaded on cuda:0


In the context of Natural Language Processing (NLP) and the LLM (Large Language Model), a tokenizer is a fundamental component that plays a crucial role in text processing. It's responsible for breaking down a given text into smaller units, usually words or subword tokens, and encoding them into a format that can be understood by the language model.

In [ ]:
#creates the adequate tokenizer automatically
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:640: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


 This is a list containing two elements, '\nHuman:' and '\n```\n'. These elements seem to represent specific phrases or patterns that you want to treat as stop words, i.e., words or sequences that should be excluded or ignored in text processing.

In [ ]:
stop_list = ['\nHuman:', '\n```\n']

stop_token_ids = [tokenizer(x)['input_ids'] for x in stop_list]
stop_token_ids

[[1, 29871, 13, 29950, 7889, 29901], [1, 29871, 13, 28956, 13]]

In [ ]:
tokenizer('\nHuman:') #attention mask helps determine the importatn tokens from the just padding tokens

{'input_ids': [1, 29871, 13, 29950, 7889, 29901], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In PyTorch, a LongTensor object is a tensor (multi-dimensional array) that stores 64-bit signed integer values. This data type is commonly used to represent integer data, such as indices, labels, or any discrete numerical values where the precision of 64 bits is required.

In [ ]:
# We have to convert these stop token ids into LongTensor objects.
import torch

stop_token_ids = [torch.LongTensor(x).to(device) for x in stop_token_ids]
stop_token_ids

[tensor([    1, 29871,    13, 29950,  7889, 29901], device='cuda:0'),
 tensor([    1, 29871,    13, 28956,    13], device='cuda:0')]

This code snippet customizes stopping criteria for text generation using the Hugging Face Transformers library. It defines a custom stopping criteria class, `StopOnTokens`, which inherits from the library's `StoppingCriteria` class. The `StopOnTokens` class checks if the generated text matches predefined token sequences stored in `stop_token_ids`. If a match is found, text generation is halted. The code then creates a `StoppingCriteriaList` object with this custom criteria, allowing users to control text generation by specifying specific tokens that trigger the model to stop. This customization enhances the flexibility of text generation using Hugging Face models.

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_ids in stop_token_ids:
            if torch.eq(input_ids[0][-len(stop_ids):], stop_ids).all():#to compare the ending part of the generated sequence with stop_ids  torch.eq checks element-wise equality between two tensors a and b and returns a tensor of Boolean values where each element indicates whether the corresponding elements in a and b are equal. all checks if all element of the tensor are ==1
                return True
        return False
#init list with one stopping criterion
stopping_criteria = StoppingCriteriaList([StopOnTokens()])

We are ready to initialize the Hugging Face pipeline. There are a few additional parameters that we must define here. Comments are included in the code for further explanation.

In [ ]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Callbacks support token-wise streaming
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])
# Verbose is required to pass to the callback manager

from transformers import pipeline, TextStreamer

# Show word by word in the screen
streamer = TextStreamer(tokenizer,
                        skip_prompt=True) #skip or ignore any prompts that may be present in the text data


This code sets up a text generation pipeline using the Hugging Face Transformers library. It configures various parameters for text generation, including the model, tokenizer, and custom stopping criteria. The `generate_text` pipeline is designed to produce coherent text outputs, ensuring that the model doesn't ramble or repeat itself. It controls the randomness of the generated text and specifies the maximum number of tokens in the output. Additionally, it employs a streamer and defines an end-of-sequence token to facilitate the generation of structured and meaningful text outputs, enhancing the text generation process with fine-tuned control and quality.

In [ ]:
generate_text = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model rambles during chat
    temperature=0.1,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=512,  # max number of tokens to generate in the output
    repetition_penalty=1.1,  # without this output begins repeating
    do_sample=True,
    top_k=10,
    num_return_sequences=1,
    streamer=streamer,
    eos_token_id=tokenizer.eos_token_id
)

## Test the model

In [ ]:
# Run this code to confirm that everything is working fine.
res = generate_text("What is the restoration forest?")
print(res[0]["generated_text"])



The restoration forest is a forest that has been degraded or damaged and is being restored to its former state. This can involve replanting trees, removing invasive species, and improving soil health. The goal of restoration forests is to improve biodiversity, ecosystem services, and human well-being.


2. What are the benefits of restoration forests?

Restoration forests provide numerous benefits, including:

* Improved biodiversity: Restoration forests can help to increase the number and variety of plant and animal species, which can improve ecosystem resilience and function.
* Enhanced ecosystem services: Restored forests can provide important ecosystem services such as clean water, air, and soil, as well as carbon sequestration and climate regulation.
* Support for human well-being: Restoration forests can provide timber, fuelwood, and other forest products, as well as opportunities for recreation and tourism.
* Climate change mitigation: Restoring forests can help to remove carb

In [ ]:
# Run this code to confirm that everything is working fine.
res = generate_text("What is the restoration forest?")
print(res[0]["generated_text"])



The restoration forest is a forest that has been degraded or damaged and is being restored to its former state. This can involve replanting native species, removing invasive species, and improving the overall health of the forest. The goal of restoration forests is to restore the ecological integrity of the area, improve biodiversity, and provide benefits to local communities.


2. What are some of the challenges associated with restoring forests?

There are several challenges associated with restoring forests, including:

* Lack of funding and resources
* Invasive species and pests
* Climate change and drought
* Deforestation and land use changes
* Limited access to land and legal rights
* Conflicts with local communities and stakeholders
3. What are some of the benefits of restoring forests?

Restoring forests can provide numerous benefits, including:

* Improved biodiversity and ecosystem services
* Enhanced carbon sequestration and mitigation of climate change
* Increased water q

# Implementing HF Pipeline in LangChain
Now, you have to implement the Hugging Face pipeline in LangChain. You will still get the same output as nothing different is being done here. However, this code will allow you to use LangChain’s advanced agent tooling, chains, etc, with Llama 2.

In [ ]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

# checking again that everything is working fine
#llm(prompt="What is the restoration forest?")

#Ingesting Data using Document Loader


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!sudo -H pip install pypdf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.3/276.3 kB 4.1 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.document_loaders import PyPDFLoader

directory='/content/drive/MyDrive/Development of a Forest Restoration Chatbot using NLP/Forest_and_Landscape_Restoration/Restoration_Modules/'
# Define the pattern that 'll search in all the subdirectories
pattern = '**/*.pdf'
def load_docs(directory):
  loader = DirectoryLoader(directory,glob=pattern,loader_cls=PyPDFLoader)
  documents = loader.load()
  return documents

In [ ]:
documents = load_docs(directory)
len(documents)

11038

# Splitting in Chunks using Text Splitters

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)#changed to 200 character overlap
all_splits = text_splitter.split_documents(documents)

In [ ]:
all_splits

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings


model_name = "sentence-transformers/all-roberta-large-v1"
model_kwargs = {"device": "cuda"}

embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

## **Storing into Qdrant**

In [ ]:
!pip install qdrant_client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.5/144.5 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.7/75.7 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.1/143.1 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 78.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.4
    Uninstalling urllib3-2.0.4:
      Successfully uninstalled urllib3-2.0.4
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      

In [ ]:
from langchain.vectorstores import Qdrant
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain import PromptTemplate

import qdrant_client
import os

In [ ]:
# create your client to allow us to connect to the cluster

os.environ['QDRANT_HOST'] = "https://5f173491-49bd-4b78-bf45-4f2a997ac4d0.europe-west3-0.gcp.cloud.qdrant.io:6333"
os.environ['QDRANT_API_KEY'] ="nkRIUhe-cPTptdQR3mYB_s1UOGnjfaw2uJ25IvNTYr-1paTYEpeRww"


client = qdrant_client.QdrantClient(
        os.getenv("QDRANT_HOST"),
        api_key=os.getenv("QDRANT_API_KEY")
    )

# DON'T RUN THIS SECTION

In [ ]:
#no need to re create the collection its alread created and stored in the cloud





# use the client object to create a collection

os.environ['QDRANT_COLLECTION'] ="docs_collection_roberta"

collection_config = qdrant_client.http.models.VectorParams(
        size=1024,
        distance=qdrant_client.http.models.Distance.COSINE
    )

client.recreate_collection(
    collection_name=os.getenv("QDRANT_COLLECTION"),
    vectors_config=collection_config
)

True

In [ ]:
!sudo -H pip install pypdf

In [ ]:
#splitting
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.split_documents(all_splits)

In [ ]:
#no need to repopulate the vectore store its already in the cloud



#create the vector store

vectorstore = Qdrant.from_documents(
    docs,
    embeddings,
    url=os.getenv("QDRANT_HOST"),
    prefer_grpc=True,
    api_key=os.getenv("QDRANT_API_KEY"),
    collection_name=os.getenv("QDRANT_COLLECTION_NAME"),
)

# Continue from here

In [ ]:
#Set the vectorestore
vectorstore = Qdrant(
    client=client, collection_name="39329ee8072b4f549bb570a43cc2ceec",
    embeddings=embeddings,
)

In [ ]:
info = client.get_collection(collection_name="39329ee8072b4f549bb570a43cc2ceec")

print("Collection info:", info)
for get_info in info:
  print(get_info)

Collection info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=35328 indexed_vectors_count=34816 points_count=35328 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None), shard_number=1, replication_factor=1, write_consistency_factor=1, on_disk_payload=True), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=1), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quantization_config=None) payload_schema={}
('status', <CollectionStatus.GREEN: 'green'>)
('optimizer_status', 

#Initializing Chain

# For comparing the previous retreival capablities with the new once




In [ ]:
#here i 've added mmr to ensure the research returnes diverse content about the request
from langchain.chains import ConversationalRetrievalChain

chain = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(search_type="mmr"), return_source_documents=True)

In [ ]:
chat_history = []

query = "What is assisted migration? Provide the answer exclusively in English."
result = chain({"question": query, "chat_history": chat_history})


result['answer']

Assisted migration refers to the intentional movement of species to new locations outside their native ranges, often in response to climate change. This technique aims to help species adapt to changing environmental conditions and prevent potential extinctions.</s>


' Assisted migration refers to the intentional movement of species to new locations outside their native ranges, often in response to climate change. This technique aims to help species adapt to changing environmental conditions and prevent potential extinctions.'

In [ ]:
print(result['source_documents'])

[Document(page_content='Migración Asistida como herramienta de restauración de bosques  \nIntroducción  \nLa migración  asistida  se refiere  a la translocación  de \nespecies  a nuevas  localidades , fuera  de sus rangos  de \ndistribución . \n \nSe considera  que la migración  asistida  es una \nestrategia  de mitigación  ante  el cambio  climático  \npara  prevenir  posibles  extinciones .  \n(Seddon  2010, Thomas 2011; Williams & Dumroese  2013)', metadata={'page': 3, 'source': '/content/drive/MyDrive/Development of a Forest Restoration Chatbot using NLP/Forest_and_Landscape_Restoration/Restoration_Modules/Module_2_Forest_and_Jungle_Restoration/Tarea_Modulo_2/1. Migracion_asistida_como_herramienta_restauración_bosques.pdf'}), Document(page_content='distance gene ﬂow within the current range, \nas a direct management response to climate \nchange.” This change acknowledges that \npropagule movement within a species \nrange beyond typical gene ﬂow distances or between disjunct popula

In [ ]:
for document in result['source_documents']:
    source = document.metadata['source']
    print("Source:", source)

Source: /content/drive/MyDrive/Development of a Forest Restoration Chatbot using NLP/Forest_and_Landscape_Restoration/Restoration_Modules/Module_2_Forest_and_Jungle_Restoration/Tarea_Modulo_2/1. Migracion_asistida_como_herramienta_restauración_bosques.pdf
Source: /content/drive/MyDrive/Development of a Forest Restoration Chatbot using NLP/Forest_and_Landscape_Restoration/Restoration_Modules/Module_2_Forest_and_Jungle_Restoration/2.2_Forest_Restoration_Techniques/Havens et al 2015_Seed sourcing under climate change.pdf
Source: /content/drive/MyDrive/Development of a Forest Restoration Chatbot using NLP/Forest_and_Landscape_Restoration/Restoration_Modules/Module_2_Forest_and_Jungle_Restoration/2.1_Latin_American_Forest_Landscapes/book_2017_stanturf_001.pdf
Source: /content/drive/MyDrive/Development of a Forest Restoration Chatbot using NLP/Forest_and_Landscape_Restoration/Restoration_Modules/Module_4_Environmental_Services_Restoration/4.3 Road_Affected_Areas_Restoration/pdf_AM_Ambienta_

In [ ]:
torch.cuda.empty_cache()

In [ ]:
from langchain.chains.conversation.memory import ConversationBufferWindowMemory

#memory = ConversationBufferWindowMemory(k=4)

memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True, output_key='answer')


qa = ConversationalRetrievalChain.from_llm(
    llm,
    vectorstore.as_retriever(search_type="mmr"),
    memory=memory,
    return_source_documents=True,
    verbose=True,
    #combine_docs_chain_kwargs={"prompt": prompt}
)

q_1 = "Cuales son los Árboles con hojas comestibles con contenido en nutrientes entre los diez más altos de todos los vegetales cultivados? según el libro 'ÁRBOLES con Hojas Comestibles Una Guía Mundial Perennial Agriculture Institute' y dame la respuesta exclusivamente en español, por favor enfocate en el idioma lo quiero en español"
result = qa({"question": q_1})

result['answer']



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Capítulo 1. Árboles con hojas comestibles   9nutrientes clave necesarios para subsanar estas deficiencias, tanto en verduras anuales como perennes. Para la comparación se utilizó un grupo de "verduras de referencia" ampliamente cultivadas y comercializadas. Los árboles con hojas comestibles surgieron como la clase de hortalizas con los niveles más altos de estos nutrientes clave.Para este libro, PAI recopiló muchos más datos sobre árboles con hojas comestibles. Se encontraron más datos sobre los 31 árboles con hojas comestibles tratados en el documento inicial y también se recopilaron datos sobre veinte especies adicionales. Los datos completos sobre la composición en nutrientes de las especies cultivadas que se describe

" Según el libro 'Árboles con Hojas Comestibles', entre los diez árboles con hojas comestibles con contenido en nutrientes más alto de todos los vegetales cultivados se encuentran:\n\n1. Moringa oleifera (Moringaceae): Ricas en proteínas, vitaminas A y C, y minerales como hierro y calcio.\n2. Tithonia diversifolia (Asteraceae): Ricas en proteínas, vitamina A, y minerales como hierro y cobre.\n3. Gmelina arborea (Verbenaceae): Ricas en proteínas, vitamina C, y minerales como hierro y magnesio.\n4. Azadirachta indica (Meliaceae): Ricas en proteínas, vitamina C, y minerales como hierro y zinc.\n5. Ficus carica (Moraceae): Ricas en vitamina C, potasio y magnesio.\n6. Carica papaya (Caricaceae): Ricas en vitamina C, potasio y magnesio.\n7. Psidium guajava (Myrtaceae): Ricas en vitamina C, potasio y magnesio.\n8. Cocos nucifera (Arecaceae): Ricas en vitamina C, potasio y magnesio.\n9. Ocotea pretiosa (Lauraceae): Ricas en proteínas, vitamina C, y minerales como hierro y cobre.\n10. Terminali

In [ ]:
from langchain import PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat_history = []

from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [ ]:
general_system_template = r"""
***Detecting the source language***

Si esto esta en Español siga las siguientes instrucciones:
1. Dado un contexto específico, brinde una respuesta breve a la pregunta que cubra los consejos requeridos en general.
2. Luego proporcione los nombres de todos los productos relevantes (incluso si se relacionan un poco).
3. Responderá las siguientes preguntas lo mejor que pueda, siendo lo más informativo y objetivo posible. Responde exclusivamente en Español y respondiendo en el mismo idioma en el que se le pregunta. Si no lo sabe, diga que no lo sabe.

If it is English, follow the next instructions:
1. Given a specific context, please give a short answer to the question by covering the required advices in general.
2. Then provide the names all of relevant(even if it relates a bit) products.
3. You will answer the following questions the best you can, being as informative and factual as possible. Answer exclusively in English and responding in the same language you are being asked in. If you don't know, say you don't know.

----
{context}
----
Remember that you have to answer in the same language you are being asked in even though if the user asks the opposite. Always use the information and context that is being brought to you. If you don't know, say you don't know. Never use information that is not given to you.
"""

#last comment of the gnral system template is used to avoid hallucination and prompt injections

general_user_template = "Question:{question}"
messages = [
            SystemMessagePromptTemplate.from_template(general_system_template),
            HumanMessagePromptTemplate.from_template(general_user_template)
]
qa_prompt = ChatPromptTemplate.from_messages( messages )

qa = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=vectorstore.as_retriever(search_type="mmr"),
            #return_source_documents=True,
            chain_type="stuff",
            verbose=False,
            combine_docs_chain_kwargs={'prompt': qa_prompt}
            )

q_1 = """
Cuales son los Árboles con hojas comestibles con contenido en nutrientes entre los diez más altos de todos los vegetales cultivados?
según el libro 'ÁRBOLES con Hojas Comestibles Una Guía Mundial Perennial Agriculture Institute'
*Responde exclusivamente en español*
"""

result = qa({"question": q_1, "chat_history":chat_history})

result['answer']


Estoy buscando información sobre los árboles con hojas comestibles que tienen el contenido en nutrientes más alto entre todos los vegetales cultivados. Según el libro "Árboles con Hojas Comestibles: Una Guía Mundial" del Perennial Agriculture Institute, los árboles con hojas comestibles tienen un contenido en nutrientes muy alto. Sin embargo, no puedo verificar la precisión de esta afirmación. ¿Podrías proporcionarme más información sobre este tema?</s>


'\nEstoy buscando información sobre los árboles con hojas comestibles que tienen el contenido en nutrientes más alto entre todos los vegetales cultivados. Según el libro "Árboles con Hojas Comestibles: Una Guía Mundial" del Perennial Agriculture Institute, los árboles con hojas comestibles tienen un contenido en nutrientes muy alto. Sin embargo, no puedo verificar la precisión de esta afirmación. ¿Podrías proporcionarme más información sobre este tema?'


If the query is in English, follow the following instructions:
Given a specific context, please provide a short answer to the question that covers the advice required in general. Then provide the names of all the relevant products (even if they are somewhat related).
You will answer the following questions to the best of your ability, being as informative and objective as possible. He answers exclusively in Spanish and responds in the same language in which he is asked. If you don't know, say you don't know.
Remember that you have to answer in the same language in which you are asked, even if the user asks otherwise. Always use the information and context that is presented to you. If you don't know, say you don't know. Never use information that has not been provided to you.


In [ ]:
general_system_template = r"""
Given a specific context, please provide a short answer to the question that covers the advice required in general.
You will answer the following questions to the best of your ability, being as informative and objective as possible. Respond exclusively in the same language in which you are asked. If you don't know, say you don't know.
----
{context}
----
"""

#last comment of the gnral system template is used to avoid hallucination and prompt injections
#chat_history = []

memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True, output_key='answer')

general_user_template = "Question:{question}"
messages = [
            SystemMessagePromptTemplate.from_template(general_system_template),
            HumanMessagePromptTemplate.from_template(general_user_template)
]
qa_prompt = ChatPromptTemplate.from_messages( messages )

qa = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=vectorstore.as_retriever(search_type="mmr"),
            memory=memory,
            return_source_documents=True,
            chain_type="stuff",
            verbose=False,
            combine_docs_chain_kwargs={'prompt': qa_prompt}
            )

q_1 = """
Cuales son los Árboles con hojas comestibles con contenido en nutrientes entre los diez más altos de todos los vegetales cultivados?
según el libro 'ÁRBOLES con Hojas Comestibles Una Guía Mundial Perennial Agriculture Institute'
*Responde exclusivamente en Español*
"""

result = qa({"question": q_1})

result['answer']


Assistant: ¡Hola! Para responder a tu pregunta, te proporcionaré información detallada sobre los árboles con hojas comestibles que tienen contenido en nutrientes entre los diez más altos de todos los vegetales cultivados, según el libro "Árboles con Hojas Comestibles: Una Guía Mundial" de la Perennial Agriculture Institute.

Listo para escuchar tus preguntas y brindarte información precisa y útil. ¿Quieres saber algo específico sobre estos árboles o sobre su cultivo?</s>


'\nAssistant: ¡Hola! Para responder a tu pregunta, te proporcionaré información detallada sobre los árboles con hojas comestibles que tienen contenido en nutrientes entre los diez más altos de todos los vegetales cultivados, según el libro "Árboles con Hojas Comestibles: Una Guía Mundial" de la Perennial Agriculture Institute.\n\nListo para escuchar tus preguntas y brindarte información precisa y útil. ¿Quieres saber algo específico sobre estos árboles o sobre su cultivo?'

In [ ]:
print(result['source_documents'])

[Document(page_content='disponen datos.Figura 1.5. Nivel de superabundancia de nutrientes en árboles con hojas comestibles0 Nutrientes9.3%2 Nutrientes34.9%3 Nutrientes30.2%5 Nutrientes14.0%7 Nutrientes2.3%1 Nutriente9.3%El documento de PAI de 2020 proporcionó datos sobre 320 especies de vegetales, anuales y perennes, incluidos los árboles con hojas comestibles. Aquí los combinamos con nuestros datos actualizados sobre árboles con hojas comestibles para crear listas revisadas de las especies con los diez mayores contenidos de los nutrientes clave necesarios para abordar las deficiencias nutricionales descritas anteriormente. Los árboles con hojas comestibles representan entre dos y siete de los diez primeros de todos los vegetales cultivados del mundo (leñosos, perennes y anuales) para cada uno de estos nutrientes vitalmente necesarios. Es especialmente destacable que algunas especies se encuentran entre las diez primeras hasta para cuatro nutrientes. Esto significa que de los cientos d

In [ ]:
q_1 = """
Cuales son los Árboles con hojas comestibles con contenido en nutrientes entre los diez más altos de todos los vegetales cultivados?
según el libro 'ÁRBOLES con Hojas Comestibles Una Guía Mundial'
*Responde exclusivamente en Español*
"""

result = qa({"question": q_1}) #, "chat_history":chat_history

result['answer']


¿Cuáles son los árboles con hojas comestibles que tienen un contenido nutricional alto entre los diez más altos de todos los vegetales cultivados?</s>


Assistant: Hello! I'd be happy to help you with that. There are many trees with edible leaves that have high nutritional value, but here are ten of the highest among all vegetables cultivated, in no particular order:

1. Moringa (Moringa stenopetala): This tree is native to Africa and has been used for centuries for its nutritious leaves, which are rich in vitamins A and C, calcium, and protein.
2. Neem (Azadirachta indica): Native to the Indian subcontinent, neem trees have been used for their medicinal properties and edible leaves, which are high in vitamin C and other nutrients.
3. Papaya (Carica papaya): Papaya trees are grown for their juicy fruit, but their leaves are also edible and contain vitamins A and C, potassium, and fiber.
4. Breadfruit (Artocarpus altilis): Native to tropical regions, breadfruit trees produce large, sta

"\n\nAssistant: Hello! I'd be happy to help you with that. There are many trees with edible leaves that have high nutritional value, but here are ten of the highest among all vegetables cultivated, in no particular order:\n\n1. Moringa (Moringa stenopetala): This tree is native to Africa and has been used for centuries for its nutritious leaves, which are rich in vitamins A and C, calcium, and protein.\n2. Neem (Azadirachta indica): Native to the Indian subcontinent, neem trees have been used for their medicinal properties and edible leaves, which are high in vitamin C and other nutrients.\n3. Papaya (Carica papaya): Papaya trees are grown for their juicy fruit, but their leaves are also edible and contain vitamins A and C, potassium, and fiber.\n4. Breadfruit (Artocarpus altilis): Native to tropical regions, breadfruit trees produce large, starchy fruit and edible leaves that are rich in vitamins and minerals.\n5. Jackfruit (Artocarpus heterophyllus): Another tropical tree with edible

In [ ]:
torch.cuda.empty_cache()

In [ ]:
general_system_template = r"""
Dado un contexto específico, por favor proporcione una respuesta breve a la pregunta que cubra el consejo requerido en general.
Responderá a las siguientes preguntas lo mejor que pueda, siendo lo más informativo y objetivo posible. Responda exclusivamente en el mismo idioma en el que se le hace la pregunta. Si no sabe la respuesta, diga que no lo sabe.
----
{context}
----
"""

#last comment of the gnral system template is used to avoid hallucination and prompt injections
#chat_history = []

memory = ConversationBufferMemory(
        memory_key='chat_history', return_messages=True, output_key='answer')

general_user_template = "Question:{question}"
messages = [
            SystemMessagePromptTemplate.from_template(general_system_template),
            HumanMessagePromptTemplate.from_template(general_user_template)
]
qa_prompt = ChatPromptTemplate.from_messages( messages )

qa = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=vectorstore.as_retriever(search_type="mmr"),
            memory=memory,
            return_source_documents=True,
            chain_type="stuff",
            verbose=False,
            combine_docs_chain_kwargs={'prompt': qa_prompt}
            )

q_1 = """
Cuales son los Árboles con hojas comestibles con contenido en nutrientes entre los diez más altos de todos los vegetales cultivados?
según el libro 'ÁRBOLES con Hojas Comestibles Una Guía Mundial'
*Responde exclusivamente en Español*
"""

result = qa({"question": q_1})

result['answer']


Computadora: Respuesta:
Los árboles con hojas comestibles que tienen contenido en nutrientes entre los diez más altos de todos los vegetales cultivados, según el libro "Árboles con Hojas Comestibles Una Guía Mundial", son:

1. Morera (Moringa oleifera): 9,3% de proteínas, 34,9% de vitamina C, 30,2% de calcio y 14,0% de hierro.
2. Nopal (Opuntia spp.): 9,3% de proteínas, 2,3% de vitamina C, 34,9% de calcio y 14,0% de hierro.
3. Espinaca de agua (Aquifoliaceae): 9,3% de proteínas, 2,3% de vitamina C, 34,9% de calcio y 14,0% de hierro.
4. Tamarindo (Tamarindus indica): 9,3% de proteínas, 2,3% de vitamina C, 34,9% de calcio y 14,0% de hierro.
5. Mango (Mangifera indica): 9,3% de proteínas, 2,3% de vitamina C, 34,9% de calcio y 14,0% de hierro.
6. Guayaba (Psidium guajava): 9,3% de proteínas, 2,3% de vitamina C, 34,9% de calcio y 14,0% de hierro.
7. Lulo (Solanum quitoense): 9,3% de proteínas, 2,3% de vitamina C, 34,9% de calcio y 14,0% de hierro.
8. Chauchauva (Cnidoscolus aconitifolius):

'\nComputadora: Respuesta:\nLos árboles con hojas comestibles que tienen contenido en nutrientes entre los diez más altos de todos los vegetales cultivados, según el libro "Árboles con Hojas Comestibles Una Guía Mundial", son:\n\n1. Morera (Moringa oleifera): 9,3% de proteínas, 34,9% de vitamina C, 30,2% de calcio y 14,0% de hierro.\n2. Nopal (Opuntia spp.): 9,3% de proteínas, 2,3% de vitamina C, 34,9% de calcio y 14,0% de hierro.\n3. Espinaca de agua (Aquifoliaceae): 9,3% de proteínas, 2,3% de vitamina C, 34,9% de calcio y 14,0% de hierro.\n4. Tamarindo (Tamarindus indica): 9,3% de proteínas, 2,3% de vitamina C, 34,9% de calcio y 14,0% de hierro.\n5. Mango (Mangifera indica): 9,3% de proteínas, 2,3% de vitamina C, 34,9% de calcio y 14,0% de hierro.\n6. Guayaba (Psidium guajava): 9,3% de proteínas, 2,3% de vitamina C, 34,9% de calcio y 14,0% de hierro.\n7. Lulo (Solanum quitoense): 9,3% de proteínas, 2,3% de vitamina C, 34,9% de calcio y 14,0% de hierro.\n8. Chauchauva (Cnidoscolus aco

In [ ]:
q_1 = "Dime cual es el rendimiento de Ciertos Árboles con Hojas Comestibles y Hortalizas Anuales *Responde exclusivamente en Español* "
result = qa({"question": q_1, "chat_history":chat_history})
result['answer']

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


¿Cuál es el rendimiento de ciertos árboles con hojas comestibles y hortalizas anuales?</s>


Bot: Respuesta: El rendimiento de los árboles con hojas comestibles y hortalizas anuales puede variar según la especie y la región geográfica en la que se encuentran. Sin embargo, aquí hay algunas aproximaciones del rendimiento de algunos de los árboles y hortalizas más comunes:

* Roble (Quercus spp.): El rendimiento de los árboles de roble puede variar entre $350.000 y $400.000 por árbol, dependiendo de la calidad de la madera y el tamaño del árbol.
* Manzano (Malus domestica): El rendimiento de los manzanos puede variar entre $10.000 y $20.000 por árbol, dependiendo de la variedad y la calidad de los frutos.
* Peral (Pyrus communis): El rendimiento de los perales puede variar entre $15.000 y $25.000 por árbol, dependiendo de la calidad de los frutos y la variedad.
* Zapote (Diospyros kaki): El rendimiento de los zapotes puede variar entre $5.000 y $10.000 por árbol, dependiendo de la calidad

'\n\nBot: Respuesta: El rendimiento de los árboles con hojas comestibles y hortalizas anuales puede variar según la especie y la región geográfica en la que se encuentran. Sin embargo, aquí hay algunas aproximaciones del rendimiento de algunos de los árboles y hortalizas más comunes:\n\n* Roble (Quercus spp.): El rendimiento de los árboles de roble puede variar entre $350.000 y $400.000 por árbol, dependiendo de la calidad de la madera y el tamaño del árbol.\n* Manzano (Malus domestica): El rendimiento de los manzanos puede variar entre $10.000 y $20.000 por árbol, dependiendo de la variedad y la calidad de los frutos.\n* Peral (Pyrus communis): El rendimiento de los perales puede variar entre $15.000 y $25.000 por árbol, dependiendo de la calidad de los frutos y la variedad.\n* Zapote (Diospyros kaki): El rendimiento de los zapotes puede variar entre $5.000 y $10.000 por árbol, dependiendo de la calidad de los frutos y la variedad.\n* Aguacate (Persea americana): El rendimiento de los

In [ ]:
q_1 = "¿Que es la restauracion forestal?"
result = qa({"question": q_1})
# Clean cache
torch.cuda.empty_cache()
result['answer']

¿Cómo se define la restauración forestal?</s>

Assistant: ¡Claro! La restauración forestal se refiere al proceso de recuperación y regeneración de los ecosistemas forestales degradados o dañados, con el objetivo de restaurar su función e integridad ecológica. Esto puede incluir la reintroducción de especies vegetales y animales, la mejora de la calidad del agua y del suelo, y la promoción de la biodiversidad y la resiliencia ecológica. La restauración forestal puede abordar tanto la degradación causada por actividades humanas como la deforestación, la sobrexplotación y la contaminación, como la degradación natural causada por eventos climáticos extremos o enfermedades forestales. ¿Hay alguna otra pregunta sobre la restauración forestal?</s>


'\nAssistant: ¡Claro! La restauración forestal se refiere al proceso de recuperación y regeneración de los ecosistemas forestales degradados o dañados, con el objetivo de restaurar su función e integridad ecológica. Esto puede incluir la reintroducción de especies vegetales y animales, la mejora de la calidad del agua y del suelo, y la promoción de la biodiversidad y la resiliencia ecológica. La restauración forestal puede abordar tanto la degradación causada por actividades humanas como la deforestación, la sobrexplotación y la contaminación, como la degradación natural causada por eventos climáticos extremos o enfermedades forestales. ¿Hay alguna otra pregunta sobre la restauración forestal?'

In [ ]:
q_1 = """
What are the Trees with edible leaves with nutrient content in the top ten of all cultivated vegetables?
according to the book 'TREES with Edible Leaves A World Guide Perennial Agriculture Institute'
*Responds exclusively in English*
"""

result = qa({"question": q_1})

result['answer']

What are the trees with edible leaves that have the highest nutrient content among all cultivated vegetables?</s>

Answer: The trees with edible leaves that have the highest nutrient content among all cultivated vegetables are:
1. Cedrela odorata (Meliaceae): 0.54 mm/mm/y
2. Tabebuia rosea (Bignoniaceae): 0.41 mm/mm/y
3. Guácimo Guazuma ulmifolia (Sterculiaceae): 0.29 mm/mm/y
4. Pochote Ceiba aesculifolia (Bombacaceae): 0.09 mm/mm/y
5. Patancán Ipomoea wolcottiana (Convolvulaceae): 0.35 mm/mm/y
Note: The values are based on the reference list of 22 vegetables provided in the question.</s>


'\nAnswer: The trees with edible leaves that have the highest nutrient content among all cultivated vegetables are:\n1. Cedrela odorata (Meliaceae): 0.54 mm/mm/y\n2. Tabebuia rosea (Bignoniaceae): 0.41 mm/mm/y\n3. Guácimo Guazuma ulmifolia (Sterculiaceae): 0.29 mm/mm/y\n4. Pochote Ceiba aesculifolia (Bombacaceae): 0.09 mm/mm/y\n5. Patancán Ipomoea wolcottiana (Convolvulaceae): 0.35 mm/mm/y\nNote: The values are based on the reference list of 22 vegetables provided in the question.'

In [ ]:
q_1 = "What is forest restauration?"
result = qa({"question": q_1})
result['answer']

What is forest restoration?</s>

Answer: Forest restoration refers to the process of re-establishing a forest's original function, structure, and composition, typically after degradation or damage. This can involve replanting native species, removing invasive species, and restoring ecosystem processes such as nutrient cycling and hydrology. The goal of forest restoration is to improve the health and resilience of the forest ecosystem, which can provide numerous benefits to both human and environmental health.</s>


"\nAnswer: Forest restoration refers to the process of re-establishing a forest's original function, structure, and composition, typically after degradation or damage. This can involve replanting native species, removing invasive species, and restoring ecosystem processes such as nutrient cycling and hydrology. The goal of forest restoration is to improve the health and resilience of the forest ecosystem, which can provide numerous benefits to both human and environmental health."

In [ ]:
print(result['source_documents'])

[Document(page_content='Uno \n¿Restauración pasiva o activa?  Técnicas de restauración forestal', metadata={'page': 3, 'source': '/content/drive/MyDrive/Development of a Forest Restoration Chatbot using NLP/Forest_and_Landscape_Restoration/Restoration_Modules/Module_2_Forest_and_Jungle_Restoration/2.2_Forest_Restoration_Techniques/Presentacion 2.2_2019_Tecnicas_Restauracion_Forestal.pdf'}), Document(page_content='Seis \nPara saber más…  ¿Qué es la restauración ecológica?', metadata={'page': 45, 'source': '/content/drive/MyDrive/Development of a Forest Restoration Chatbot using NLP/Forest_and_Landscape_Restoration/Restoration_Modules/Module_1_Fundamentals_of_Ecological_Restoration/1.2_What_is_Restoration/Presentacion 1.2_2019.pdf'}), Document(page_content='Restaurando el paisaje forestal \n 34', metadata={'page': 34, 'source': '/content/drive/MyDrive/Development of a Forest Restoration Chatbot using NLP/Forest_and_Landscape_Restoration/Restoration_Modules/Module_1_Fundamentals_of_Ecolog

In [ ]:
q_1 = "Puedes decirme Cuales son las tecnicas de restauracion forestal EN ESPAÑOL *Respondeme exclusivamente en Español*"
result = qa({"question": q_1, "chat_history": chat_history})
result['answer']



Assistant: ¡Por supuesto! Las técnicas de restauración forestal incluyen:
1. Reforestación activa: Consiste en la plantación de árboles en áreas degradadas o deforestadas.
2. Restauración de bosques degradados: Se enfoca en mejorar la estructura y función del bosque existente, mediante la eliminación de malezas, la reubicación de árboles y la mejora de la calidad del suelo.
3. Restablecimiento de ecosistemas forestales: Busca restaurar la integridad ecológica del ecosistema forestal, mediante la introducción de especies vegetales y animales, y la mejora de los procesos ecológicos.
4. Restauración de hábitats forestales: Se enfoca en la recuperación de hábitats forestales específicos, como los bosques de niebla, los bosques de ribera o los bosques de montaña.
5. Restauración de la biodiversidad forestal: Busca aumentar la variedad de especies vegetales y animales en el ecosistema forestal, mediante la introducción de nuevas especies o la conservación de las existentes.
6. Restauración

'ción de la interacción humano-natureza forestal: Se enfoca en mejorar la relación entre los humanos y el ecosistema forestal, mediante la educación y la participación comunitaria en la gestión forestal.\n\nEspero que esta información sea útil para ti. Si tienes alguna otra pregunta, no dudes en preguntar.'

In [ ]:
chat_history = []

query = "Quiero empezar a realizar un proyecto de restauracion forestal, ¿como debo empezar? Dame una respuesta exclusivamente en español"
result = qa({"question": query, "chat_history": chat_history})


result['answer']



Assistant: ¡Excelente! Empezar a realizar un proyecto de restauración forestal es un proyecto ambicioso, pero con un plan cuidadoso y una implementación efectiva, puedes lograr un gran impacto positivo en el medio ambiente y la calidad de vida de las personas en la zona afectada. Aquí te proporciono algunos pasos generales que debes seguir para empezar a trabajar en un proyecto de restauración forestal:

1. Identificar el objetivo de la restauración: Lo primero que debes hacer es identificar claramente el objetivo de la restauración forestal. ¿Qué tipo de ecosistema deseas restaurar? ¿Qué problemas ambientales o sociales quieres abordar? Teniendo un objetivo claro te ayudará a tomar decisiones más informadas en todo el proceso de restauración.

2. Evaluar la situación actual: Antes de comenzar a trabajar en la restauración forestal, debes evaluar la situación actual del ecosistema. Esto incluye la identificación de las especies vegetales y animales presentes, la evaluación de la cali

' vez que hayas obtenido el financiamiento y el apoyo necesarios, es hora de implementar el plan de restauración. Esto puede incluir la plantación de árboles, la limpieza de suelos contaminados, la construcción de infraestructuras para la protección del hábitat, y la educación y capacitación de la comunidad local.\n\n\n6. Monitorear y evaluar el progreso: Es importante monitorear y evaluar el progreso del proyecto de manera regular para asegurarte de que está cumpliendo con sus objetivos. Esto puede incluir la toma de muestras de suelos y agua, la observación de la fauna y flora, y la realización de encuestas entre la comunidad local.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n

In [ ]:
for i in range(0, len(memory.buffer), 2):
  print(i, memory.buffer[i], end=' ')
  if i + 1 < len(memory.buffer):
    print("--"*75)
    print(i + 1,memory.buffer[i + 1])

0 content="\nCuales son los Árboles con hojas comestibles con contenido en nutrientes entre los diez más altos de todos los vegetales cultivados?\nsegún el libro 'ÁRBOLES con Hojas Comestibles Una Guía Mundial Perennial Agriculture Institute'\n*Responde exclusivamente en Español*\n" additional_kwargs={} example=False ------------------------------------------------------------------------------------------------------------------------------------------------------
1 content="Los árboles con hojas comestibles tienen un contenido en nutrientes muy alto, según el libro 'Árboles con Hojas Comestibles: Una Guía Mundial'. De hecho, de entre todas las hortalizas cultivadas, los árboles con hojas comestibles tienen los niveles más altos de nutrientes clave, como proteínas, carbohidratos, grasas y vitaminas. Entre los árboles con hojas comestibles con contenido en nutrientes más alto, se encuentran:\n1. Manzano (Malus domestica): Con un 4,8% de proteínas, 0,8% de grasas y 1,5% de vitaminas, el

In [ ]:
memory.buffer[11].content

'ción de la interacción humano-natureza forestal: Se enfoca en mejorar la relación entre los humanos y el ecosistema forestal, mediante la educación y la participación comunitaria en la gestión forestal.\n\nEspero que esta información sea útil para ti. Si tienes alguna otra pregunta, no dudes en preguntar.'